In [2]:
!pip install pymongo

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached pymongo-3.11.4-cp37-cp37m-manylinux2014_x86_64.whl (512 kB)


In [53]:
import pymongo
import pandas as pd
import numpy as np
import pickle
#import tensorflow as tf
from datetime import datetime, timedelta
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.model_selection import LeaveOneOut, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import sklearn
 
def corregir_dtypes_dataframe(df):
    df.cantidad = pd.to_numeric(df.cantidad)
    return df
 
def crear_dataframe_desde_mongodb():
    cadena_conexion = ('mongodb://{nom_usuario}:{password}@{host}:{port}')
    cadena_conexion = cadena_conexion.format(
        nom_usuario = 'PY01_c02',
        password = 'P4rd83XkXrTz',
        host = '5.189.129.12',
        port = 27017
    )
    client = pymongo.MongoClient(cadena_conexion)
    db = client.PY01
    header = [*db.PY01.find_one().keys()]
    lista_dataset = list()
    for doc in db.PY01.find():
        lista_dataset.append([*doc.values()])
 
    df = pd.DataFrame(data = lista_dataset, columns = header)
    return df
 
def crear_codigos_prod():
    productos = {0:'KS_302',1:'KS_304',2:'KS_307',3:'KS_308',4:'KS_309',5:'KS_310',6:'KS_311',7:'KS_312',8:'KS_313',9:'KS_318',10:'KS_319',11:'KS_320',12:'KS_321',13:'KS_324',14:'KS_325',15:'KS_326',16:'KS_328',17:'KS_329',18:'KS_330',19:'KS_331',20:'KS_332',21:'KS_333',22:'KS_335',23:'KS_336',24:'KS_337',25:'KS_338',26:'KS_339',27:'KS_340',28:'KS_341',29:'KS_342',30:'KS_343',31:'KS_344',32:'KS_345',33:'KS_346',34:'KS_348',35:'KS_349',36:'KS_350',37:'KS_352',38:'KS_355',39:'KS_358',40:'KS_359',41:'KS_360',42:'KS_361',43:'KS_362',44:'KS_364',45:'KS_365',46:'KS_366',47:'KS_367',48:'KS_368',49:'KS_369',50:'KS_370',51:'KS_371',52:'KS_372',53:'KS_373',54:'KS_374',55:'KS_378',56:'KS_379',57:'KS_381',58:'KS_382',59:'KS_383',60:'KS_386',61:'KS_387',62:'KS_388',63:'KS_389',64:'KS_394',65:'KS_396',66:'KS_398',67:'KS_399',68:'KS_402',69:'KS_404',70:'KS_406',71:'KS_407',72:'KS_408',73:'KS_409',74:'KS_410',75:'KS_414',76:'KS_415',77:'KS_422',78:'KS_423',79:'KS_442',80:'KS_443',81:'KS_454',82:'KS_456',83:'KS_458',84:'KS_460',85:'KS_461',86:'KS_462',87:'KS_464',88:'KS_465',89:'KS_467',90:'KS_468',91:'KS_469',92:'KS_478',93:'KS_479',94:'KS_484',95:'KS_485',96:'KS_486',97:'KS_487',98:'KS_488',99:'KS_489',100:'KS_490',101:'KS_491',102:'KS_494',103:'KS_495',104:'KS_496',105:'KS_497',106:'KS_511',107:'KS_516',108:'KS_517',109:'KS_518',110:'KS_526',111:'KS_527',112:'KS_532',113:'KS_533',114:'KS_536',115:'KS_537',116:'KS_538',117:'KS_539',118:'KS_540',119:'KS_542',120:'KS_543',121:'KS_544',122:'KS_545',123:'KS_546',124:'KS_547',125:'KS_548',126:'KS_549',127:'KS_551',128:'KS_552',129:'KS_555',130:'KS_556',131:'KS_558',132:'KS_559',133:'KS_560',134:'KS_561',135:'KS_562',136:'KS_564',137:'KS_566',138:'KS_568',139:'KS_570',140:'KS_571',141:'KS_572',142:'KS_574',143:'KS_575',144:'KS_576',145:'KS_578',146:'KS_579',147:'KS_582',148:'KS_583',149:'KS_584',150:'KS_585',151:'KS_586',152:'KS_587',153:'KS_590',154:'KS_591',155:'KS_592',156:'KS_594',157:'KS_595',158:'KS_596',159:'KS_598',160:'KS_600',161:'KS_602',162:'KS_603',163:'KS_604',164:'KS_605',165:'KS_606',166:'KS_607',167:'KS_610',168:'KS_612',169:'KS_613',170:'KS_614',171:'KS_615',172:'KS_624',173:'KS_626',174:'KS_635',175:'KS_636',176:'KS_638',177:'KS_644',178:'KS_645',179:'KS_649',180:'KS_651',181:'KS_652',182:'KS_653',183:'KS_654',184:'KS_655',185:'KS_656',186:'KS_657',187:'KS_661',188:'KS_662',189:'KS_664',190:'KS_665',191:'KS_666',192:'KS_668',193:'KS_669',194:'KS_676',195:'KS_680',196:'KS_681',197:'KS_682',198:'KS_688',199:'KS_689',200:'KS_690',201:'KS_692',202:'KS_693',203:'KS_696',204:'KS_702',205:'KS_704',206:'KS_711',207:'KS_712',208:'KS_713',209:'KS_718',210:'KS_719',211:'KS_720',212:'KS_721',213:'KS_938',214:'KS_303',215:'KS_305',216:'KS_306',217:'KS_314',218:'KS_315',219:'KS_327',220:'KS_347',221:'KS_353',222:'KS_354',223:'KS_363',224:'KS_375',225:'KS_376',226:'KS_377',227:'KS_384',228:'KS_390',229:'KS_395',230:'KS_397',231:'KS_400',232:'KS_403',233:'KS_405',234:'KS_411',235:'KS_416',236:'KS_417',237:'KS_418',238:'KS_420',239:'KS_424',240:'KS_426',241:'KS_427',242:'KS_430',243:'KS_431',244:'KS_432',245:'KS_433',246:'KS_434',247:'KS_435',248:'KS_437',249:'KS_438',250:'KS_440',251:'KS_441',252:'KS_444',253:'KS_446',254:'KS_447',255:'KS_448',256:'KS_450',257:'KS_452',258:'KS_453',259:'KS_457',260:'KS_459',261:'KS_463',262:'KS_466',263:'KS_472',264:'KS_473',265:'KS_474',266:'KS_475',267:'KS_476',268:'KS_492',269:'KS_493',270:'KS_502',271:'KS_503',272:'KS_504',273:'KS_505',274:'KS_506',275:'KS_508',276:'KS_509',277:'KS_510',278:'KS_512',279:'KS_514',280:'KS_519',281:'KS_524',282:'KS_528',283:'KS_529',284:'KS_530',285:'KS_541',286:'KS_550',287:'KS_553',288:'KS_554',289:'KS_557',290:'KS_563',291:'KS_567',292:'KS_573',293:'KS_577',294:'KS_580',295:'KS_593',296:'KS_597',297:'KS_599',298:'KS_601',299:'KS_608',300:'KS_611',301:'KS_616',302:'KS_620',303:'KS_625',304:'KS_627',305:'KS_630',306:'KS_632',307:'KS_633',308:'KS_634',309:'KS_640',310:'KS_648',311:'KS_650',312:'KS_660',313:'KS_663',314:'KS_667',315:'KS_670',316:'KS_672',317:'KS_673',318:'KS_674',319:'KS_675',320:'KS_677',321:'KS_678',322:'KS_679',323:'KS_691',324:'KS_697',325:'KS_698',326:'KS_703',327:'KS_705',328:'KS_708',329:'KS_710',330:'KS_722',331:'KS_724',332:'KS_939',333:'KS_941',334:'KS_942',335:'KS_944',336:'KS_945',337:'KS_947',338:'KS_948',339:'KS_950',340:'KS_953',341:'KS_956',342:'KS_959',343:'KS_960',344:'KS_962',345:'KS_963',346:'KS_965',347:'KS_966',348:'KS_968',349:'KS_969',350:'KS_971',351:'KS_974',352:'KS_975',353:'KS_981',354:'KS_982',355:'KS_990',356:'KS_994',357:'KS_995',358:'KS_997',359:'KS_1000',360:'KS_1003',361:'KS_1006',362:'KS_1009',363:'KS_1010',364:'KS_1012',365:'KS_1016',366:'KS_1018',367:'KS_1019',368:'KS_1024',369:'KS_1030',370:'KS_1033',371:'KS_1036',372:'KS_1037',373:'KS_1045',374:'KS_1048',375:'KS_1051'}
    return productos
 
def limpiar_dataset(df):
    dias = {'LUNES': 1, 'MARTES': 2, 'MIERCOLES': 3, 'JUEVES': 4, 'VIERNES': 5, 'SABADO': 6, 'SUNDAY': 7, 'DOMINGO': 7}
    for dia, cod in dias.items():
        df.loc[df.dia == dia, 'dia'] = cod
    df.dia = pd.to_numeric(df.dia)
    
    for cod, prod in crear_codigos_prod().items():
        df.loc[df['prod'] == prod, 'prod'] = cod
    return df
 
def generar_variables(df):
    nuevos_registros = list()
    for row in df.itertuples():
        cantidad = row.cantidad
        if cantidad > 1:
            cantidad -= 1
            for t in range(cantidad):
                nuevos_registros.append(row)
 
    df = df.append(nuevos_registros, ignore_index = True).drop(labels = ['Index','cantidad'], axis = 1)
 
    df = pd.concat((df, pd.get_dummies(df.dia, prefix = 'dia')), axis = 1)
    dias_presentes = df.dia.unique()
    if 1 not in dias_presentes:
        df = df.assign(dia_1 = 0)
    if 2 not in dias_presentes:
        df = df.assign(dia_2 = 0)
    if 3 not in dias_presentes:
        df = df.assign(dia_3 = 0)
    if 4 not in dias_presentes:
        df = df.assign(dia_4 = 0)
    if 5 not in dias_presentes:
        df = df.assign(dia_5 = 0)
    if 6 not in dias_presentes:
        df = df.assign(dia_6 = 0)
    if 7 not in dias_presentes:
        df = df.assign(dia_7 = 0)
    df = df.assign(dia_sin = lambda x: np.sin(2 * np.pi * x.dia / 7))
    df = df.assign(dia_cos = lambda x: np.cos(2 * np.pi * x.dia / 7))
 
    return df[['cliente','dia','dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','dia_sin','dia_cos','prod']] 
 
def preparar_dataset(df):
    df = df.drop(labels = ['_id','fecNacimiento','sexo','anho','mes','monto','descuento'], axis = 1)
    df = corregir_dtypes_dataframe(df)
    df = limpiar_dataset(df)
    df = generar_variables(df)
    return df
 
def generar_lista_variantes():
    variantes = {1: ['dia','prod'],
                 2: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','prod'],
                 3: ['dia_sin','dia_cos','prod']}
    return variantes
 
def aplicar_grid_search(X, y, modelo, params):
    if len(X) > 100:
        cv = StratifiedKFold(10)
    else:
        cv = LeaveOneOut()
    gs = GridSearchCV(estimator = modelo,
                      param_grid = params,
                      cv = cv,
                      scoring = ['accuracy','f1_weighted'],
                      refit = 'f1_weighted',
                      return_train_score = True,
                      verbose = 2,
                      n_jobs = -1)
    #with tf.device('/device:GPU:0'):
    gs.fit(X, y)
    return gs
 
def entrenar_LR(metodo_multiclass, X, y):
    modelo = LogisticRegression()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__C = np.logspace(-6,2,9), estimator__solver = ('newton-cg','sag','saga','lbfgs'))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_SVM(metodo_multiclass, X, y):
    modelo = SVC()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__C = np.logspace(-6,2,9), estimator__gamma = np.logspace(-6,2,9))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_DT(metodo_multiclass, X, y):
    modelo = DecisionTreeClassifier()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__max_depth = range(2,11,2), estimator__min_samples_split = range(2,11,2))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_NB(metodo_multiclass, X, y):
    modelo = GaussianNB()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__var_smoothing = np.logspace(-11,-7,5))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_KNN(metodo_multiclass, X, y):
    modelo = KNeighborsClassifier()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__n_neighbors = range(2, 31, 3), estimator__weights = ('uniform','distance'), estimator__p = (1,2))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar(df, variantes):
    lista_clientes = sorted(df.cliente.unique(), key = lambda x: int(x[3:]))
    lista_clientes.remove('SID1')
    modelo_cliente = list()
    for cliente in lista_clientes:
        print(cliente)
        clasificadores = list()
        for cod_variante, variante in variantes.items():
            base = df[df.cliente == cliente][variante].copy()
            X = base.drop(labels = 'prod', axis = 1)
            y = base['prod'].astype('int')
            if len(y.unique()) > 1:
                for metodo_multiclass in (OneVsOneClassifier, OneVsRestClassifier):
                    for entrenar_algoritmo in (entrenar_NB, entrenar_DT, entrenar_LR, entrenar_SVM):
                        clasificadores.append((cliente, entrenar_algoritmo(metodo_multiclass, X, y)))
            else:
                clasificadores.append((cliente, y.iloc[0]))
        modelo_cliente.append(clasificadores)
    return modelo_cliente
 
def main():
    df = crear_dataframe_desde_mongodb()
    df = preparar_dataset(df)
    variantes = generar_lista_variantes()
    c = entrenar(df, variantes)
    return c
 
np.random.seed(10)
#ans = main()

In [ ]:
import pickle
pickle_out = open('trabajo.pkl', 'wb')
pickle.dump(ans, pickle_out)
pickle_out.close()

In [1]:
!pip install scikit-learn==0.23.2 --upgrade

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
import pickle
pickle_in = open('pickles/trabajo.pkl', 'rb')
ans = pickle.load(pickle_in)
pickle_in.close()

In [18]:
from sklearn.model_selection import GridSearchCV
clientes_cod = {1:'SID2',2:'SID3',3:'SID5',4:'SID6',5:'SID7',6:'SID8',7:'SID9',8:'SID10',9:'SID12',10:'SID13',11:'SID14',12:'SID15',13:'SID17',14:'SID18',15:'SID20',16:'SID21',17:'SID22',18:'SID23',19:'SID24',20:'SID25',21:'SID26',22:'SID27',23:'SID28',24:'SID29',25:'SID30',26:'SID32',27:'SID33',28:'SID34',29:'SID35',30:'SID36',31:'SID38',32:'SID39',33:'SID40',34:'SID41',35:'SID42',36:'SID43',37:'SID44',38:'SID45',39:'SID46',40:'SID47',41:'SID48',42:'SID49',43:'SID50',44:'SID52',45:'SID53',46:'SID54',47:'SID55',48:'SID56',49:'SID57',50:'SID59',51:'SID61',52:'SID65',53:'SID66',54:'SID67',55:'SID68',56:'SID70',57:'SID71',58:'SID72',59:'SID73',60:'SID74',61:'SID75',62:'SID76',63:'SID77',64:'SID78',65:'SID79',66:'SID80',67:'SID81',68:'SID82',69:'SID83',70:'SID84',71:'SID85',72:'SID86',73:'SID87',74:'SID88',75:'SID89',76:'SID90',77:'SID91',78:'SID92',79:'SID93'}
n_features_variantes = {1: 1, 7: 2, 2: 3}
scores_keys = ('mean_train_accuracy','std_train_accuracy','mean_test_accuracy','std_test_accuracy','mean_train_f1_weighted','std_train_f1_weighted','mean_test_f1_weighted','std_test_f1_weighted')
clasificadores = dict()
for cod_cliente, cliente in enumerate(ans):
    print(clientes_cod[cod_cliente + 1])
    if type(cliente[0]) == GridSearchCV:
        mejor_gs = sorted(cliente, key = lambda gs: (gs.best_score_, gs.cv_results_['mean_test_accuracy'][gs.best_index_], gs.cv_results_['mean_train_f1_weighted'][gs.best_index_], gs.cv_results_['mean_train_accuracy'][gs.best_index_]), reverse = True)[0]
        mejor_clasificador = mejor_gs.best_estimator_
        variante = n_features_variantes[mejor_clasificador.n_features_in_]
        scores = {x: mejor_gs.cv_results_[x][mejor_gs.best_index_] for x in scores_keys if x in mejor_gs.cv_results_}
        cv = mejor_gs.cv
        n_classes = len(mejor_gs.classes_)
    else:
        mejor_clasificador = cliente[0]
        variante = None
        scores = None
        cv = None
        n_classes = 1
    clasificadores[clientes_cod[cod_cliente + 1]] = (variante, mejor_clasificador, scores, cv, n_classes)
    

SID2
SID3
SID5
SID6
SID7
SID8
SID9
SID10
SID12
SID13
SID14
SID15
SID17
SID18
SID20
SID21
SID22
SID23
SID24
SID25
SID26
SID27
SID28
SID29
SID30
SID32
SID33
SID34
SID35
SID36
SID38
SID39
SID40
SID41
SID42
SID43
SID44
SID45
SID46
SID47
SID48
SID49
SID50
SID52
SID53
SID54
SID55
SID56
SID57
SID59
SID61
SID65
SID66
SID67
SID68
SID70
SID71
SID72
SID73
SID74
SID75
SID76
SID77
SID78
SID79
SID80
SID81
SID82
SID83
SID84
SID85
SID86
SID87
SID88
SID89
SID90
SID91
SID92
SID93


In [17]:
mejor_gs.n_splits_

5

In [10]:
mejor_gs.cv

LeaveOneOut()

In [19]:
clasificadores

{'SID2': (3,
  OneVsOneClassifier(estimator=LogisticRegression(C=100.0, solver='newton-cg')),
  {'mean_train_accuracy': 0.45454545454545453,
   'std_train_accuracy': 0.015560299929124038,
   'mean_test_accuracy': 0.45454545454545453,
   'std_test_accuracy': 0.49792959773196915,
   'mean_train_f1_weighted': 0.3660629796720776,
   'std_train_f1_weighted': 0.01640830654167225,
   'mean_test_f1_weighted': 0.45454545454545453,
   'std_test_f1_weighted': 0.49792959773196915},
  LeaveOneOut(),
  12),
 'SID3': (3,
  OneVsRestClassifier(estimator=LogisticRegression(C=1e-06, solver='saga')),
  {'mean_train_accuracy': 0.37499999999999994,
   'std_train_accuracy': 0.06916041689656102,
   'mean_test_accuracy': 0.375,
   'std_test_accuracy': 0.4841229182759271,
   'mean_train_f1_weighted': 0.20833333333333331,
   'std_train_f1_weighted': 0.06301282428353334,
   'mean_test_f1_weighted': 0.375,
   'std_test_f1_weighted': 0.4841229182759271},
  LeaveOneOut(),
  4),
 'SID5': (1,
  OneVsOneClassifier(est

In [20]:
pickle_out = open('pickles/clasificadores_v2.pkl', 'wb')
pickle.dump(clasificadores, pickle_out)
pickle_out.close()

In [16]:
!pip install scikit-learn==0.23.2

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached scikit_learn-0.23.2-cp37-cp37m-manylinux1_x86_64.whl (6.8 MB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.1
    Uninstalling scikit-learn-0.22.1:
      Successfully uninstalled scikit-learn-0.22.1


In [17]:
pickle_in = open('pickles/clasificadores.pkl', 'rb')
clasificadores = pickle.load(pickle_in)
pickle_in.close()

In [55]:
from sklearn.metrics import f1_score

df = crear_dataframe_desde_mongodb()
df = df.drop(labels = ['_id','fecNacimiento','sexo','monto','descuento'], axis = 1)

train = df[((df.anho == '2020') & (df.mes == 'Julio'))]
train = train.drop(labels = ['anho','mes'], axis = 1)
train = corregir_dtypes_dataframe(train)
train = limpiar_dataset(train)
train = generar_variables(train)

test = df[((df.anho == '2021') & (df.mes == 'Mayo'))]
test = test.drop(labels = ['anho','mes'], axis = 1)
test = corregir_dtypes_dataframe(test)
test = limpiar_dataset(test)
test = generar_variables(test)

#clasificadores = obtener_clasificadores()
variantes = generar_lista_variantes()

curvas_por_cliente = dict()
for cliente, (cod_variante, clasificador) in clasificadores.items():
    print(cliente, cod_variante)
    if cod_variante is None: continue
    train_cliente = train[train.cliente == cliente][variantes[cod_variante]]
    X_train_cliente = train_cliente.drop(labels = 'prod', axis = 1)
    y_train_cliente = train_cliente['prod'].astype('int')
    
    test_cliente = test[test.cliente == cliente][variantes[cod_variante]]
    if len(test_cliente) == 0: continue
    X_test_cliente = test_cliente.drop(labels = 'prod', axis = 1)
    y_test_cliente = test_cliente['prod'].astype('int')
    
    cantidad_training_samples = len(X_train_cliente)
    curva_aprendizaje = list()
    for train_samples in range(cantidad_training_samples):
        train_samples += 1
        if train_samples == 1: continue
        print(train_samples)
        clasificador.fit(X_train_cliente.head(train_samples), y_train_cliente.head(train_samples))
        y_train_pred = clasificador.predict(X_train_cliente)
        y_test_pred = clasificador.predict(X_test_cliente)
        score_train = f1_score(y_train_cliente, y_train_pred, average = 'weighted')
        score_test = f1_score(y_test_cliente, y_test_pred, average = 'weighted')
        curva_aprendizaje.append((train_samples, (score_train, score_test), (y_train_pred, y_test_pred, y_test_cliente)))
    curvas_por_cliente[cliente] = curva_aprendizaje

SID2 3
SID3 3
SID5 1
SID6 1
SID7 1
SID8 1
SID9 3
SID10 2
SID12 1
SID13 1
2
3
4
5
6
7
8
SID14 2
SID15 1
SID17 1
SID18 1
SID20 None
SID21 1
SID22 1
SID23 1
2
SID24 1
SID25 1
SID26 1
SID27 1
SID28 None
SID29 1
SID30 1
2
3
4
5
6
SID32 None
SID33 None
SID34 3
SID35 None
SID36 None
SID38 1
SID39 1
SID40 1
SID41 2
SID42 1
SID43 1
SID44 1
SID45 None
SID46 1
SID47 1
SID48 1
SID49 None
SID50 None
SID52 None
SID53 1
SID54 None
SID55 1
2
3
SID56 None
SID57 None
SID59 1
SID61 1
SID65 None
SID66 1
SID67 None
SID68 1
SID70 None
SID71 None
SID72 1
SID73 None
SID74 1
SID75 1
SID76 None
SID77 1
SID78 1
SID79 1
SID80 1
SID81 1
SID82 None
SID83 None
SID84 None
SID85 1
SID86 None
SID87 1
SID88 None
SID89 1
SID90 None
SID91 1
SID92 None
SID93 None


/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in true_divide
  joint_log_likelihood = []
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in true_divide
  joint_log_likelihood = []
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in true_divide
  joint_log_likelihood = []
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in subtract
  joint_log_likelihood = []
/opt/conda/lib/python3.7/site-pack

In [56]:
productos = crear_codigos_prod()

for cliente, v in curvas_por_cliente.items():
    print(cliente)
    for e in v:
        print(e[0], e[1], [productos[p] for p in e[2][1]], [productos[p] for p in e[2][2]])

SID13
2 (0.6428571428571428, 0.0) ['KS_644'] ['KS_536']
3 (0.6428571428571428, 0.0) ['KS_644'] ['KS_536']
4 (0.6428571428571428, 0.0) ['KS_644'] ['KS_536']
5 (0.6428571428571428, 0.0) ['KS_644'] ['KS_536']
6 (0.6428571428571428, 0.0) ['KS_644'] ['KS_536']
7 (0.6428571428571428, 0.0) ['KS_644'] ['KS_536']
8 (0.6428571428571428, 0.0) ['KS_644'] ['KS_536']
SID23
2 (1.0, 0.0) ['KS_688', 'KS_688', 'KS_688', 'KS_688', 'KS_688', 'KS_688'] ['KS_327', 'KS_1037', 'KS_1037', 'KS_1037', 'KS_1037', 'KS_1037']
SID30
2 (0.3333333333333333, 0.0) ['KS_337', 'KS_337', 'KS_337'] ['KS_369', 'KS_369', 'KS_369']
3 (0.3333333333333333, 0.0) ['KS_337', 'KS_337', 'KS_337'] ['KS_369', 'KS_369', 'KS_369']
4 (0.3333333333333333, 0.0) ['KS_337', 'KS_337', 'KS_337'] ['KS_369', 'KS_369', 'KS_369']
5 (0.3333333333333333, 0.0) ['KS_337', 'KS_337', 'KS_337'] ['KS_369', 'KS_369', 'KS_369']
6 (0.3333333333333333, 0.0) ['KS_337', 'KS_337', 'KS_337'] ['KS_369', 'KS_369', 'KS_369']
SID55
2 (0.16666666666666666, 0.0666666666

In [58]:
curvas_por_cliente

{'SID13': [(2,
   (0.6428571428571428, 0.0),
   (array([177, 177, 177, 177, 177, 177, 177, 177]),
    array([177]),
    340    114
    Name: prod, dtype: int64)),
  (3,
   (0.6428571428571428, 0.0),
   (array([177, 177, 177, 177, 177, 177, 177, 177]),
    array([177]),
    340    114
    Name: prod, dtype: int64)),
  (4,
   (0.6428571428571428, 0.0),
   (array([177, 177, 177, 177, 177, 177, 177, 177]),
    array([177]),
    340    114
    Name: prod, dtype: int64)),
  (5,
   (0.6428571428571428, 0.0),
   (array([177, 177, 177, 177, 177, 177, 177, 177]),
    array([177]),
    340    114
    Name: prod, dtype: int64)),
  (6,
   (0.6428571428571428, 0.0),
   (array([177, 177, 177, 177, 177, 177, 177, 177]),
    array([177]),
    340    114
    Name: prod, dtype: int64)),
  (7,
   (0.6428571428571428, 0.0),
   (array([177, 177, 177, 177, 177, 177, 177, 177]),
    array([177]),
    340    114
    Name: prod, dtype: int64)),
  (8,
   (0.6428571428571428, 0.0),
   (array([177, 177, 177, 177, 

In [57]:
pickle_out = open('pickles/validacion.pkl', 'wb')
pickle.dump(curvas_por_cliente, pickle_out)
pickle_out.close()